In [33]:
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np

# Load your trained model
model = load_model("../models/lstm_model.h5")
print("Model loaded successfully.")


Model loaded successfully.


In [128]:
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load model
model = load_model("../models/lstm_model.h5")
print("✅ Model loaded successfully.")

# Load tokenizer
with open("../models/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

# Load label encoder
with open("../models/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# Load dataset
df_model = pd.read_csv("../data/processed/t20s_combined.csv")
df_model = df_model.dropna(subset=['wicket_type'])

# Rebuild input 'text' column with labels
df_model['text'] = (
    "Batsman: " + df_model['striker'].astype(str) +
    " | Non-striker: " + df_model['non_striker'].astype(str) +
    " | Bowler: " + df_model['bowler'].astype(str) +
    " | Batting Team: " + df_model['batting_team'].astype(str) +
    " | Bowling Team: " + df_model['bowling_team'].astype(str) +
    " | Runs off bat: " + df_model['runs_off_bat'].astype(str) +
    " | Over: " + df_model['ball'].astype(str) +                             
    " | Venue: " + df_model['venue'].astype(str)                           
)



# Prepare padded sequences using loaded tokenizer
texts = df_model['text'].tolist()
sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences)

# Encode labels using loaded label encoder
labels = label_encoder.transform(df_model['wicket_type'].tolist())


✅ Model loaded successfully.


C:\Users\sundu\AppData\Local\Temp\ipykernel_29244\401185788.py:20: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_model = pd.read_csv("../data/processed/t20s_combined.csv")


In [137]:
# =============================
# 1. Define strategy function
# =============================
def get_strategy_suggestion(wicket_type):
    suggestions = {
        'caught': "Expect aggressive fielders at deep mid-wicket and long-on.",
        'bowled': "Bowler targeting stumps – keep length full and straight.",
        'run out': "Fielding pressure working – increase in-fielders near circle.",
        'lbw': "Attack the pads – spinner or yorker could work.",
        'hit wicket': "Rare dismissal – bowler exploiting batter’s footwork.",
        'stumped': "Wicketkeeper sharp – time to dive from spinners.",
        'retired hurt': "Injury concern – team might slow down pace.",
        'retired not out': "Unusual case – monitor batter’s intent.",
        'obstructing the field': "Rare call – tight coordination among fielders.",
        'handled the ball': "Unlikely dismissal – watch batter discipline.",
        'caught and bowled': "Sharp reflexes – keep bowler alert on follow-through."
    }
    return suggestions.get(wicket_type, "No specific strategy available.")

# =============================
# 2. Split train/test from full padded data
# =============================
X = x  # already tokenized + padded
y = labels  # encoded labels

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# =============================
# 3. Sample and Predict
# =============================
import random
import numpy as np

sample_indices = random.sample(range(len(X_test)), 5)
sample_inputs = X_test[sample_indices]

sample_probs = model.predict(sample_inputs)
sample_preds = np.argmax(sample_probs, axis=1)
sample_labels = label_encoder.inverse_transform(sample_preds)

# =============================
# 4. Decode inputs back to text
# =============================
sample_texts = df_model['text'].iloc[sample_indices].tolist()


# =============================
# 5. Display in DataFrame
# =============================
import pandas as pd
results = []
for i in range(5):
    results.append({
        'Input Context': sample_texts[i],
        'Predicted Wicket Type': sample_labels[i],
        'Strategy Suggestion': get_strategy_suggestion(sample_labels[i])
    })

pd.set_option('display.max_colwidth', None)
output_df = pd.DataFrame(results)
output_df


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


,Input Context,Predicted Wicket Type,Strategy Suggestion
0,"Batsman: BKG Mendis | Non-striker: LD Chandimal | Bowler: K Rabada | Batting Team: Sri Lanka | Bowling Team: South Africa | Runs off bat: 0 | Over: 0.7 | Venue: R.Premadasa Stadium, Khettarama",caught,Expect aggressive fielders at deep mid-wicket and long-on.
1,Batsman: T Bavuma | Non-striker: Q de Kock | Bowler: SM Curran | Batting Team: South Africa | Bowling Team: England | Runs off bat: 0 | Over: 0.6 | Venue: Newlands,run out,Fielding pressure working – increase in-fielders near circle.
2,Batsman: Shafali Verma | Non-striker: S Mandhana | Bowler: S Ecclestone | Batting Team: India | Bowling Team: England | Runs off bat: 0 | Over: 5.5 | Venue: Junction Oval,bowled,Bowler targeting stumps – keep length full and straight.
3,Batsman: PWH de Silva | Non-striker: DM de Silva | Bowler: SN Thakur | Batting Team: Sri Lanka | Bowling Team: India | Runs off bat: 0 | Over: 13.6 | Venue: Maharashtra Cricket Association Stadium,caught,Expect aggressive fielders at deep mid-wicket and long-on.
4,Batsman: KJ O'Brien | Non-striker: GJ Delany | Bowler: D Ajekun | Batting Team: Ireland | Bowling Team: Nigeria | Runs off bat: 0 | Over: 5.2 | Venue: Sheikh Zayed Stadium,caught,Expect aggressive fielders at deep mid-wicket and long-on.
